# Sentiment Analysis of The Times Music Reviews
## Part III: Data Visualization
*How have artforms been reported?  Is there a status hierarchy between them?  How has this changed over time?*

* **Project:** What counts as culture?  Reporting and criticism in The Times 1785-2000
* **Project Lead:** Dave O'Brien
* **Developer:** Lucy Havens
* **Funding:** from the Centre for Data, Culture & Society, University of Edinburgh
* **Dataset:** 83,625 reviews about music published in The Times from 1950 through 2009

Begun February 2021

***

First, import required programming libraries.

In [1]:
# For data loading
import re
import string
import numpy as np
import pandas as pd

# For text analysis
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
# nltk.download('punkt')
from nltk.corpus import PlaintextCorpusReader
# nltk.download('wordnet')
from nltk.corpus import wordnet
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.text import Text
# nltk.download('averaged_perceptron_tagger')
# nltk.download('tagsets')  # part of speech tags
from nltk.tag import pos_tag

# For data visualization
import matplotlib.pyplot as plt
import altair as alt
import seaborn as sn